In [1]:
import os
from pathlib import Path
from lxml import etree as ET
from abc import ABC, abstractmethod 

### Contants

In [2]:
CURR_WORK_DIR = Path.cwd()
ROOT_NAME = 'news_agency'

### Relevant code

In [66]:
class XmlElement(ABC):
    @property
    @abstractmethod
    def tag(self):
        pass

    @abstractmethod
    def to_xml_element(self):
        pass
    
    @abstractmethod
    def from_xml_element(self):
        pass
    

class Category(XmlElement):
    tag = 'category'

    def __init__(self, id_: int, name: str):
        self.id = id_
        self.name = name
    
    def to_xml_element(self):
        return ET.Element(_tag=Category.tag, id=str(self.id), name=self.name)
    
    @classmethod
    def from_xml_element(cls, element):
        return cls(id_=element.attrib['id'],name=element.attrib['name'])
    
    def __repr__(self):
        return f'{type(self).__name__}(id_={self.id!r}, name={self.name!r})'
    

class News(XmlElement):
    tag = 'news'

    def __init__(self, id_: int, text: str, category: Category):
        self.id = id_
        self.text = text
        self.category = category
    
    def to_xml_element(self):
        element = ET.Element(_tag=News.tag, id=str(self.id))
        element.text=self.text
        return element
    
    @classmethod
    def from_xml_element(cls, element, category):
        return cls(id_=element.attrib['id'],name=element.attrib['name'],category=category)
    
    def __repr__(self):
        return f'{type(self).__name__}(id_={self.id!r}, text={self.text!r}, category={self.category!r})'

In [67]:
class XlmController:
    def __init__(self, categories=None, news=None):
        self.categories = categories or []
        self.news = news or []

    def validate(self):
        return (len(set([category.id for category in self.categories])) == len(
            self.categories)
                and len(set([news_item.id
                             for news_item in self.news])) == len(self.news))

    def write(self, path_to_file):
        if not self.validate():
            raise Exception(
                'Cant write data to xml file because some values have the same ids.'
            )

        path_to_file = Path(path_to_file)
        path_to_file.parent.mkdir(parents=True, exist_ok=True)
        path_to_file.touch(exist_ok=True)

        root = ET.Element(ROOT_NAME)

        categories = {}
        for category in self.categories:
            xml_category = category.to_xml_element()
            categories[category.id] = xml_category
            root.append(xml_category)

        for news_item in news:
            parent = categories[news_item.category.id]
            parent.append(news_item.to_xml_element())

        ET.ElementTree(root).write(file=str(path_to_file), pretty_print=True)
    
    @classmethod
    def read(cls, path_to_file):
        context = ET.parse(str(path_to_file))
        
        news = []
        categories = []
        for category in context.getroot():
            categories.append(Category(id_=category.attrib['id'],name=category.attrib['name']))
            for news_item in category:
                news.append(News(id_=news_item.attrib['id'],text=news_item.text,category=categories[-1]))
        return cls(categories=categories, news=news)

### Usage example

In [70]:
categories = [Category(id_=1,name='name1'),Category(id_=2,name='name2')]
news = [News(id_=1,text='news1',category=categories[0]),News(id_=2,text='news2',category=categories[0])]
controller = XlmController(categories=categories,news=news)
controller.write(CURR_WORK_DIR / 'filename.xml')
controller.validate()
read_res =XlmController.read(str(CURR_WORK_DIR / 'filename.xml'))
res.news[0]

News(id_='1', text='news1', category=Category(id_='1', name='name1'))